In [2]:
import folium
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from folium.plugins import HeatMap
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [3]:
crimenes_cod = {
    "ROBO": 1, "ASALTO": 2, "VANDALISMO": 3, "FRAUDE": 4, "VIOLENCIA DOMÉSTICA": 5,
    "HURTO": 6, "SECUESTRO": 7, "TRÁFICO DE DROGAS": 8, "HOMICIDIO": 9, "ESTAFA": 10,
    "ACOSO SEXUAL": 11, "EXTORSIÓN": 12, "CONTRABANDO": 13, "DELITO INFORMÁTICO": 14
}

provincia_codigo = {
    'SAN JOSÉ': 1, 'ALAJUELA': 2, 'CARTAGO': 3, 'HEREDIA': 4, 
    'GUANACASTE': 5, 'PUNTARENAS': 6, 'LIMÓN': 7
}

cantones_dict = {
    'SAN JOSÉ': 1.1, 'ESCAZÚ': 1.2, 'DESAMPARADOS': 1.3, 'PURISCAL': 1.4, 'TARRAZÚ': 1.5, 'ASERRÍ': 1.6,
    'MORA': 1.7, 'GOICOECHEA': 1.8, 'SANTA ANA': 1.9, 'ALAJUELITA': 1.10, 'VÁZQUEZ DE CORONADO': 1.11,
    'ACOSTA': 1.12, 'TIBÁS': 1.13, 'MORAVIA': 1.14, 'MONTES DE OCA': 1.15, 'TURRUBARES': 1.16, 
    'DOTA': 1.17, 'CURRIDABAT': 1.18, 'PÉREZ ZELEDÓN': 1.19, 'LEÓN CORTÉS': 1.20,
    
    'ALAJUELA': 2.1, 'SAN RAMÓN': 2.2, 'GRECIA': 2.3, 'SAN MATEO': 2.4, 'ATENAS': 2.5, 'NARANJO': 2.6,
    'PALMARES': 2.7, 'POÁS': 2.8, 'OROTINA': 2.9, 'SAN CARLOS': 2.10, 'ZARCERO': 2.11, 
    'VALVERDE VEGA': 2.12, 'UPALA': 2.13, 'LOS CHILES': 2.14, 'GUATUSO': 2.15, 'RÍO CUARTO': 2.16,
    
    'CARTAGO': 3.1, 'PARAÍSO': 3.2, 'LA UNIÓN': 3.3, 'JIMÉNEZ': 3.4, 'TURRIALBA': 3.5, 
    'ALVARADO': 3.6, 'OREAMUNO': 3.7, 'EL GUARCO': 3.8,
    
    'HEREDIA': 4.1, 'BARVA': 4.2, 'SANTO DOMINGO': 4.3, 'SANTA BÁRBARA': 4.4, 'SAN RAFAEL': 4.5,
    'SAN ISIDRO': 4.6, 'BELÉN': 4.7, 'FLORES': 4.8, 'SAN PABLO': 4.9, 'SARAPIQUÍ': 4.10,
    
    'LIBERIA': 5.1, 'NICOYA': 5.2, 'SANTA CRUZ': 5.3, 'BAGACES': 5.4, 'CARRILLO': 5.5, 
    'CAÑAS': 5.6, 'ABANGARES': 5.7, 'TILARÁN': 5.8, 'NANDAYURE': 5.9, 'LA CRUZ': 5.10, 
    'HOJANCHA': 5.11,
    
    'PUNTARENAS': 6.1, 'ESPARZA': 6.2, 'BUENOS AIRES': 6.3, 'MONTES DE ORO': 6.4, 'OSA': 6.5,
    'QUEPOS': 6.6, 'GOLFITO': 6.7, 'COTO BRUS': 6.8, 'PARRITA': 6.9, 'CORREDORES': 6.10, 
    'GARABITO': 6.11,
    
    'LIMÓN': 7.1, 'POCOCÍ': 7.2, 'SIQUIRRES': 7.3, 'TALAMANCA': 7.4, 'MATINA': 7.5, 'GUÁCIMO': 7.6
}

In [4]:
processed_chunks = []

for chunk in pd.read_csv('C:/Users/allan/src/projects/clustering/map/delitos_2023.csv', low_memory=False, chunksize=10000):
    if not set(['Fecha', 'Hora', 'Delito', 'Provincia', 'Cantón', 'Estado']).issubset(chunk.columns):
        print("Las columnas requeridas no están en el fragmento.")
        continue

    location = chunk.copy()  
    fecha_actual = pd.to_datetime(datetime.now()).date()

    location['fechas_cod'] = pd.to_datetime(location['Fecha'], errors='coerce')
    location = location[location['fechas_cod'].dt.date <= fecha_actual]
    location['horas_cod'] = pd.to_datetime(location['Hora'], format='%H:%M:%S', errors='coerce').dt.hour

    location['delito_cod'] = location['Delito'].map(crimenes_cod)
    location['provincia_cod'] = location['Provincia'].map(provincia_codigo)

    def codificar_canton(row):
        return cantones_dict.get(row['Cantón'], np.nan) if row['provincia_cod'] in provincia_codigo.values() else np.nan
    
    location['canton'] = location.apply(codificar_canton, axis=1)

    location = pd.get_dummies(location, columns=['Estado'], prefix='estado')

    location['horas_cod'] = location['horas_cod'].fillna(location['horas_cod'].mode()[0])
    location['delito_cod'] = location['delito_cod'].fillna(-1)
    location['provincia_cod'] = location['provincia_cod'].fillna(-1)
    location['canton'] = location['canton'].fillna(-1)

    location.dropna(subset=['fechas_cod'], inplace=True)

    location = location.drop(columns=['Fecha', 'Hora', 'Delito', 'Provincia', 'Cantón'], axis=1)

    processed_chunks.append(location)

final_location_var = pd.concat(processed_chunks, ignore_index=True)
final_location_var['dia_semana'] = final_location_var['fechas_cod'].dt.dayofweek  # Lunes=0, Domingo=6
final_location_var['mes'] = final_location_var['fechas_cod'].dt.month
final_location_var.head()


,Latitud,Longitud,fechas_cod,horas_cod,delito_cod,provincia_cod,canton,estado_No Resuelto,estado_Resuelto,dia_semana,mes
0,9.869214,-83.911605,2023-04-15,20,3,3,3.10,False,True,5,4
1,10.673962,-84.822352,2021-06-08,19,5,2,2.15,True,False,1,6
2,10.066566,-83.291348,2020-07-04,18,3,7,7.50,False,True,5,7
3,10.003581,-84.129076,2020-07-16,12,5,4,4.10,False,True,3,7
4,10.213839,-83.686139,2022-01-18,22,1,7,7.60,True,False,1,1


In [5]:
def crear_heatmap_con_predicciones(provincia=None, canton=None, hora=None, rango_horas=None):
    filtrados = final_location_var.copy()
    
    if provincia is not None:
        filtrados = filtrados[filtrados['provincia_cod'] == provincia]
    
    if canton is not None:
        filtrados = filtrados[filtrados['canton'] == canton]

    if hora is not None:
        filtrados = filtrados[filtrados['horas_cod'] == hora]
    elif rango_horas is not None:

        if isinstance(rango_horas, tuple) and len(rango_horas) == 2:
            inicio, fin = rango_horas
            filtrados = filtrados[(filtrados['horas_cod'] >= inicio) & (filtrados['horas_cod'] <= fin)]
        else:
            print("El rango de horas debe ser una tupla con dos elementos: (inicio, fin).")
            return None

    if filtrados.empty:
        print("No hay datos para las condiciones especificadas.")
        return None

    mapa = folium.Map(location=[9.9333, -84.0833], zoom_start=8)

    if 'Latitud' not in filtrados.columns or 'Longitud' not in filtrados.columns:
        print("Las columnas de 'latitud' y 'longitud' no existen en los datos.")
        return None

    ubicaciones = filtrados[['Latitud', 'Longitud']].dropna()
    
    if 'delito_cod' not in filtrados.columns:
        print("No existe la columna 'delito_cod' para hacer predicciones.")
        return None
    
    y = filtrados['delito_cod'].dropna().values  
    
    if len(ubicaciones) >= 2:
        X = ubicaciones.values  

        modelo = LinearRegression()
        modelo.fit(X, y)

        predicciones = modelo.predict(X)

        HeatMap(data=list(zip(ubicaciones['Latitud'], ubicaciones['Longitud'], predicciones)),
                 radius=20, gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'}).add_to(mapa)

    return mapa  

# Ejemplo de uso:
mapa = crear_heatmap_con_predicciones(provincia=3, canton=3.1, rango_horas=(18, 18))
mapa